In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [43]:
## Extracting features of 2022 movies from Wikipedia

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [4]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [5]:
tables = soup.find_all('table',class_='wikitable sortable')

In [6]:
len(tables)

4

In [7]:
type(tables[0])

bs4.element.Tag

In [8]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [9]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [10]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5],NaN
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN
...,...,...,...,...,...,...,...
324,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[261]
325,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN
326,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN
327,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_2022 = df[['Title','Cast and crew']]

In [13]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,American Siege,Edward Drake (director/screenplay); Timothy V....
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
...,...,...
324,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...
325,NaN,NaN
326,NaN,NaN
327,NaN,NaN


In [14]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'ff3132c29c92e21e14539982f56e664a'

In [15]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [16]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\2369738557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))


In [17]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy Horror
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Adventure Thriller
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...
324,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Mystery Drama
325,NaN,NaN,Drama
326,NaN,NaN,Drama
327,NaN,NaN,Drama


In [18]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [19]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\3796821595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


In [20]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [21]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\2929607607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [22]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [23]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\2384233428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [24]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [25]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [26]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Adventure Thriller,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
...,...,...,...,...,...,...,...
324,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Mystery Drama,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick
325,NaN,NaN,Drama,nan,nan,NaN,NaN
326,NaN,NaN,Drama,nan,nan,NaN,NaN
327,NaN,NaN,Drama,nan,nan,NaN,NaN


In [27]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [28]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [29]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN,Action Adventure Thriller,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
...,...,...,...,...,...,...
324,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Mystery Drama,"Alice, Darling"
325,nan,nan,NaN,NaN,Drama,NaN
326,nan,nan,NaN,NaN,Drama,NaN
327,nan,nan,NaN,NaN,Drama,NaN


In [30]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [31]:
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      9
actor_3_name     35
genres            1
movie_title       4
comb             36
dtype: int64

In [32]:
new_df22 = new_df22.dropna(how='any')

In [34]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [35]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\2732328091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [36]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
5,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Comedy Family Fantasy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
6,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
320,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
321,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
322,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
323,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [37]:
old_df = pd.read_csv('datasets/final_data.csv')

In [38]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5879,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
5880,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5881,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5882,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [40]:
final_df = old_df.append(new_df22,ignore_index=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_18732\1166909226.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df22,ignore_index=True)


In [41]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6172,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
6173,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
6174,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
6175,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [42]:
final_df.to_csv('main_data.csv',index=False)